In [54]:
# pip install pillow

In [55]:
import cv2
import os
from PIL import Image
import numpy as np

In [56]:
data=[]
paths=[]
result=[]

In [57]:
for r, d,f in os.walk(r'D:\DL\brain tumor detection\yes'):
    for file in f:
        if '.jpg' in file:
            paths.append(os.path.join(r,file))
            
            
for path in paths:
    img=Image.open(path)
    img=img.resize((64,64))
    img=np.array(img)
    if (img.shape==(64,64,3)):
        data.append(np.array(img))
        result.append(1)
    

In [58]:
paths=[]
for r, d,f in os.walk(r'D:\DL\brain tumor detection\no'):
    for file in f:
        if '.jpg' in file:
            paths.append(os.path.join(r,file))
            
            
for path in paths:
    img=Image.open(path)
    img=img.resize((64,64))
    img=np.array(img)
    if (img.shape==(64,64,3)):
        data.append(np.array(img))
        result.append(0)

In [59]:
data=np.array(data) #Reshape=(n,image_width,image_height,n_channel)
data.shape

(2891, 64, 64, 3)

In [60]:
result=np.array(result)
print(result)

[1 1 1 ... 0 0 0]


In [61]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(data,result,test_size=0.25,random_state=0)

In [62]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(2168, 64, 64, 3)
(723, 64, 64, 3)
(2168,)
(723,)


In [63]:
import tensorflow as tf
from tensorflow import keras
from keras.utils import normalize
from keras.utils import to_categorical

x_train=normalize(x_train,axis=1)
x_test=normalize(x_test,axis=1)

y_train=to_categorical(y_train,num_classes=2)
y_test=to_categorical(y_test,num_classes=2)

In [64]:
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Activation,Dropout,Flatten,Dense

In [65]:
model=Sequential()
model.add(Conv2D(32,(3,3),input_shape=(64,64,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32,(3,3),kernel_initializer='he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3,3),kernel_initializer='he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(2))
model.add(Activation('softmax'))


In [66]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [67]:
model.fit(x_train,y_train,batch_size=25,verbose=1,epochs=15,
         validation_data=(x_test,y_test),shuffle=False)

Epoch 1/15
87/87 [==============================] - 4s 39ms/step - loss: 0.5461 - accuracy: 0.7279 - val_loss: 0.4604 - val_accuracy: 0.7939
Epoch 2/15
87/87 [==============================] - 3s 37ms/step - loss: 0.4340 - accuracy: 0.8086 - val_loss: 0.3756 - val_accuracy: 0.8299
Epoch 3/15
87/87 [==============================] - 3s 37ms/step - loss: 0.3356 - accuracy: 0.8570 - val_loss: 0.3084 - val_accuracy: 0.8672
Epoch 4/15
87/87 [==============================] - 3s 31ms/step - loss: 0.2751 - accuracy: 0.8888 - val_loss: 0.2424 - val_accuracy: 0.9073
Epoch 5/15
87/87 [==============================] - 3s 40ms/step - loss: 0.1950 - accuracy: 0.9253 - val_loss: 0.1885 - val_accuracy: 0.9267
Epoch 6/15
87/87 [==============================] - 3s 39ms/step - loss: 0.1326 - accuracy: 0.9571 - val_loss: 0.1410 - val_accuracy: 0.9544
Epoch 7/15
87/87 [==============================] - 3s 37ms/step - loss: 0.0941 - accuracy: 0.9691 - val_loss: 0.1140 - val_accuracy: 0.9571
Epoch 8/15
87

In [68]:
from PIL import Image
image1=cv2.imread('D:\\DL\\brain tumor detection\\no\\no8.jpg')

In [69]:
img=Image.fromarray(image1)
img=img.resize((64,64))
img=np.array(img)
input_img=np.expand_dims(img,axis=0)

In [70]:
output=model.predict(input_img)
output=np.argmax(output)
print(output)
if output==0:
    print("The MRI scan is healthy")        
else:
     print("The MRI scan has a brain tumor")

1/1 [==============================] - 0s 43ms/step
0
The MRI scan is healthy


In [71]:
model.save('BrainTumorcategorical.h5')

In [72]:
from keras.models import load_model
model=load_model('BrainTumorcategorical.h5')

In [73]:
image1=cv2.imread('D:\\DL\\brain tumor detection\\yes\\y10.jpg')
img=Image.fromarray(image1)
img=img.resize((64,64))
img=np.array(img)
input_img=np.expand_dims(img,axis=0)
output=model.predict(input_img)
output=np.argmax(output)
print(output)
if output==0:
    print("The MRI scan is healthy")        
else:
     print("The MRI scan has a brain tumor")

1/1 [==============================] - 0s 41ms/step
1
The MRI scan has a brain tumor
